In [ ]:
import random
from typing import Optional
from transformers import AutoTokenizer
import os

import streamlit as st
from openai import OpenAI

# tokenizer = AutoTokenizer.from_pretrained("./tokenizer", trust_remote_code=True)

BASE_URLS = {
    "plamo2-8b-translation": os.getenv("PLAMO_TRANSLATE_BASE_URL"),
    "plamo2-8b-translation-sft": os.getenv("PLAMO_TRANSLATE_BASE_URL"),
}

MODEL_NAMES = {
    "plamo2-8b-translation": "plamo-2-8b-translation-32k",
    "plamo2-8b-translation-sft": "plamo-2-8b-translation-32k",
}

In [4]:
def translate(
    text: str,
    model_name: str,
    base_url: str,
    input_lang: str = "Japanese|English",
    output_lang: str = "Japanese|English",
    temperature: float = 0.0,
    best_of: int = 1,
    type: Optional[str] = None,
    stream: bool = False,
):
    client = OpenAI(base_url=base_url, api_key="a")
    is_started = False
    output_lang_ditected = ""

    if output_lang == "Japanese|English":
        output_lang = " lang="
    else:
        output_lang_ditected = output_lang
        output_lang = f" lang={output_lang}\n"
        is_started = True
    if type is not None:
        type = f" style={type}"
    else:
        type = ""
    messages = [
        f"""<|plamo:op|>dataset
translation
<|plamo:op|>input{type} lang={input_lang}
{text}
<|plamo:op|>output{output_lang}"""
    ]
    output = ""
    # n_tokens = len(tokenizer.encode(messages[0]))
    # max_new_tokens = 30000 - n_tokens

    if stream:
        try:
            for x in client.completions.create(
                prompt=messages,  # type: ignore
                model=model_name,
                temperature=temperature,
                max_tokens=15000,
                stop=["<|plamo:op|>"],
                stream=True,
            ):
                output += x.choices[0].text
                if not is_started and "\n" in output:
                    is_started = True
                    output_lang_ditected, output = output.split("\n", 1)
                yield output, output_lang_ditected
        except Exception as e:
            print(f"Error: {e}")
            yield f"Error: {e}", ""
    else:
        try:
            response = client.completions.create(
                prompt=messages[0],
                model=model_name,
                temperature=temperature,
                max_tokens=15000,
                stop=["<|plamo:op|>"],
            )
            output = response.choices[0].text
            if not is_started and "\n" in output:
                output_lang_ditected, output = output.split("\n", 1)
            return output, output_lang_ditected
        except Exception as e:
            print(f"Error: {e}")
            return f"Error: {e}", ""

translate(
    text="こんにちは",
    model_name=MODEL_NAMES["plamo2-8b-translation-sft"],
    base_url=BASE_URLS["plamo2-8b-translation-sft"],
)

<generator object translate at 0x12f0b36f0>

In [ ]:
client = OpenAI(base_url=BASE_URLS["plamo2-8b-translation-sft"], api_key="a")
is_started = False
output_lang_ditected = ""

if type is not None:
    type = f" style={type}"
else:
    type = ""


In [ ]:
text = "hello. how are you?"
messages = [
    f"""<|plamo:op|>dataset
translation
<|plamo:op|>input{type} lang=Japanese|English
{text}
<|plamo:op|>output lang=Japanese|English"""
]
output = ""

response = client.completions.create(
    prompt=messages[0],
    model=MODEL_NAMES["plamo2-8b-translation-sft"],
    temperature=0.,
    max_tokens=15000,
    stop=["<|plamo:op|>"],
)

''

In [17]:
response

Completion(id='cmpl-58661e28-8194-48e3-b077-ba63a105d6e7', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\nこんにちは。お元気ですか？\n', stop_reason='<|plamo:op|>', prompt_logprobs=None)], created=1748767802, model='plamo-2-8b-translation-32k', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=37, total_tokens=43, completion_tokens_details=None, prompt_tokens_details=None))

In [18]:
output = response.choices[0].text
if not is_started and "\n" in output:
    output_lang_ditected, output = output.split("\n", 1)
    
output_lang_ditected, output

('', 'こんにちは。お元気ですか？\n')

In [19]:
response.choices[0].text

'\nこんにちは。お元気ですか？\n'